## Training the model on input features that have a AUC of more than 0.60 and studying the change in efficiency



In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import uproot
import tqdm
import seaborn as sns

Processing only 30% of the data 

In [ ]:
df_list=[]
for filenumber in tqdm.tqdm(range(1,37)):
# Open the .root file
    filePath=f'D:/Physics/Project/SEM 4 PROJECT/36_files/JetNtuple_RunIISummer16_13TeV_MC_{filenumber}.root'
    file = uproot.open(filePath)
# Access the tree structu
    tree = file['AK4jets/jetTree']
# Define the columns you want to read
    columns = ['isPhysG', 'isPhysUDS','QG_mult','QG_ptD','QG_axis2','jetPt','jetEta','jetQGl','jetMass','jetGirth','jetArea','jetChargedHadronMult','jetNeutralHadronMult']
    df=tree.arrays(columns, library='pd')
    df_list.append(df)



In [ ]:
df=pd.concat(df_list,ignore_index='true')

In [ ]:
# First remove all the other types of jets from the data except the gluon and the light quark (UDS) jets
df = df[(df.isPhysG==1) | (df.isPhysUDS==1)].reset_index()
# We keep only jets that either stem from QCD or UDS.

Data in the range $0<|\eta|<2.5$ and $30$ GeV $ < p_T < 600 $ GeV. 
We select jets with jetPt with values between 30 GeV to 600 GeV to stay out of the boosted jet regime.

In [ ]:
#selecting only a range of pt 
df_pt_ranged=df[(df.jetPt>30) & (df.jetPt<600)]


In [ ]:
df_selected=df_pt_ranged

### Splitting the dataset into test and train subsets 

In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df_selected, test_size=0.2, random_state=42)

train_y = train.isPhysUDS
test_y = test.isPhysUDS

In [ ]:
# training and test variables for each training variable
input_features = ['QG_mult','QG_ptD','QG_axis2','jetPt','jetGirth','jetChargedHadronMult']

for input in input_features:
    train_x = train[[input]]
    test_x = test[[input]]
 # Normalize the features
    train_x_mean = train_x.mean()
    train_x_std = train_x.std()
    exec(f'train_x_{input}= (train_x - train_x_mean) / train_x_std')
    exec(f'test_x_{input} = (test_x - train_x_mean) / train_x_std')

train_x=train[input_features]
test_x=test[input_features]

Using a sequential model from the tensorflow library.


In [ ]:

import tensorflow as tf
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Dropout
from sklearn.utils import class_weight
from keras.callbacks import EarlyStopping


Training independant models on each of the input features and one with all the input variables.


In [ ]:

# Define the DNN architecture
model_ptD = Sequential()
model_ptD.add(Dense(100, kernel_initializer='normal', activation='relu', input_dim=train_x_QG_ptD.shape[1]))
model_ptD.add(Dropout(0.2))
model_ptD.add(Dense(100, kernel_initializer='normal', activation='relu'))
model_ptD.add(Dropout(0.2))
model_ptD.add(Dense(50, kernel_initializer='normal', activation='relu'))
model_ptD.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
model_ptD.compile(optimizer='Nadam', loss='binary_crossentropy', metrics=['accuracy'])

# Weight the training samples so that there is equal weight on gluon and quark jets
# even if there are different amount of them in the training set
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(train_y), y=train_y[:])
print(class_weights)

callback=EarlyStopping(monitor='val_loss',min_delta=0.00001,patience=10,verbose=1,mode='auto',baseline=None,restore_best_weights=False)

# Train the model
model_ptD.fit(train_x_QG_ptD,
        train_y,
        epochs=200,
        batch_size=128,
        class_weight={0:class_weights[0] , 1: class_weights[1]},
        validation_split=0.2,
        shuffle=True,
       verbose=1,callbacks=callback);

In [ ]:
model_ptD.save('models/model_ptD.h5')
model_ptD.save_weights('models/model_ptD_weights.h5')

In [ ]:

# Define the DNN architecture
model_axis2 = Sequential()
model_axis2.add(Dense(100, kernel_initializer='normal', activation='relu', input_dim=train_x_QG_axis2.shape[1]))
model_axis2.add(Dropout(0.2))
model_axis2.add(Dense(100, kernel_initializer='normal', activation='relu'))
model_axis2.add(Dropout(0.2))
model_axis2.add(Dense(50, kernel_initializer='normal', activation='relu'))
model_axis2.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
model_axis2.compile(optimizer='Nadam', loss='binary_crossentropy', metrics=['accuracy'])

# Weight the training samples so that there is equal weight on gluon and quark jets
# even if there are different amount of them in the training set
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(train_y), y=train_y[:])
print(class_weights)

callback=EarlyStopping(monitor='val_loss',min_delta=0.00001,patience=8,verbose=1,mode='auto',baseline=None,restore_best_weights=False)

# Train the model
model_axis2.fit(train_x_QG_axis2,
        train_y,
        epochs=200,
        batch_size=128,
        class_weight={0:class_weights[0] , 1: class_weights[1]},
        validation_split=0.2,
        shuffle=True,
       verbose=1,callbacks=callback)

In [ ]:
model_axis2.save('models/model_axis2.h5')
model_axis2.save_weights('models/model_axis2_weights.h5')

In [ ]:

# Define the DNN architecture
model_mult = Sequential()
model_mult.add(Dense(100, kernel_initializer='normal', activation='relu', input_dim=train_x_QG_mult.shape[1]))
model_mult.add(Dropout(0.2))
model_mult.add(Dense(100, kernel_initializer='normal', activation='relu'))
model_mult.add(Dropout(0.2))
model_mult.add(Dense(50, kernel_initializer='normal', activation='relu'))
model_mult.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
model_mult.compile(optimizer='Nadam', loss='binary_crossentropy', metrics=['accuracy'])

# Weight the training samples so that there is equal weight on gluon and quark jets
# even if there are different amount of them in the training set
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(train_y), y=train_y[:])
print(class_weights)

callback=EarlyStopping(monitor='val_loss',min_delta=0.00001,patience=8,verbose=1,mode='auto',baseline=None,restore_best_weights=False)

# Train the model
model_mult.fit(train_x_QG_mult,
        train_y,
        epochs=200,
        batch_size=128,
        class_weight={0:class_weights[0] , 1: class_weights[1]},
        validation_split=0.2,
        shuffle=True,
       verbose=1,callbacks=callback);

In [ ]:
model_mult.save('models/model_mult.h5')
model_mult.save_weights('models/model_mult_weights.h5')

In [ ]:

# Define the DNN architecture
model_jetPt = Sequential()
model_jetPt.add(Dense(100, kernel_initializer='normal', activation='relu', input_dim=train_x_QG_mult.shape[1]))
model_jetPt.add(Dropout(0.2))
model_jetPt.add(Dense(100, kernel_initializer='normal', activation='relu'))
model_jetPt.add(Dropout(0.2))
model_jetPt.add(Dense(50, kernel_initializer='normal', activation='relu'))
model_jetPt.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
model_jetPt.compile(optimizer='Nadam', loss='binary_crossentropy', metrics=['accuracy'])

# Weight the training samples so that there is equal weight on gluon and quark jets
# even if there are different amount of them in the training set
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(train_y), y=train_y[:])
print(class_weights)

callback=EarlyStopping(monitor='val_loss',min_delta=0.00001,patience=8,verbose=1,mode='auto',baseline=None,restore_best_weights=False)

# Train the model
model_jetPt.fit(train_x_jetPt,
        train_y,
        epochs=200,
        batch_size=128,
        class_weight={0:class_weights[0] , 1: class_weights[1]},
        validation_split=0.2,
        shuffle=True,
       verbose=1,callbacks=callback);

In [ ]:
model_jetPt.save('models/model_jetPt.h5')
model_jetPt.save_weights('models/model_mult_weights.h5')

In [ ]:
# Define the DNN architecture
model_jetGirth = Sequential()
model_jetGirth.add(Dense(100, kernel_initializer='normal', activation='relu', input_dim=train_x_QG_mult.shape[1]))
model_jetGirth.add(Dropout(0.2))
model_jetGirth.add(Dense(100, kernel_initializer='normal', activation='relu'))
model_jetGirth.add(Dropout(0.2))
model_jetGirth.add(Dense(50, kernel_initializer='normal', activation='relu'))
model_jetGirth.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
model_jetGirth.compile(optimizer='Nadam', loss='binary_crossentropy', metrics=['accuracy'])

# Weight the training samples so that there is equal weight on gluon and quark jets
# even if there are different amount of them in the training set
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(train_y), y=train_y[:])
print(class_weights)

callback=EarlyStopping(monitor='val_loss',min_delta=0.00001,patience=8,verbose=1,mode='auto',baseline=None,restore_best_weights=False)

# Train the model
model_jetGirth.fit(train_x_jetGirth,
        train_y,
        epochs=200,
        batch_size=128,
        class_weight={0:class_weights[0] , 1: class_weights[1]},
        validation_split=0.2,
        shuffle=True,
       verbose=1,callbacks=callback);


In [ ]:
model_jetGirth.save('models/model_jetGirth.h5')
model_jetGirth.save_weights('models/model_mult_weights.h5')

In [ ]:
# Define the DNN architecture
model_jetChargedHadronMult = Sequential()
model_jetChargedHadronMult.add(Dense(100, kernel_initializer='normal', activation='relu', input_dim=train_x_QG_mult.shape[1]))
model_jetChargedHadronMult.add(Dropout(0.2))
model_jetChargedHadronMult.add(Dense(100, kernel_initializer='normal', activation='relu'))
model_jetChargedHadronMult.add(Dropout(0.2))
model_jetChargedHadronMult.add(Dense(50, kernel_initializer='normal', activation='relu'))
model_jetChargedHadronMult.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
model_jetChargedHadronMult.compile(optimizer='Nadam', loss='binary_crossentropy', metrics=['accuracy'])

# Weight the training samples so that there is equal weight on gluon and quark jets
# even if there are different amount of them in the training set
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(train_y), y=train_y[:])
print(class_weights)

callback=EarlyStopping(monitor='val_loss',min_delta=0.00001,patience=8,verbose=1,mode='auto',baseline=None,restore_best_weights=False)

# Train the model
model_jetChargedHadronMult.fit(train_x_jetChargedHadronMult,
        train_y,
        epochs=200,
        batch_size=128,
        class_weight={0:class_weights[0] , 1: class_weights[1]},
        validation_split=0.2,
        shuffle=True,
       verbose=1,callbacks=callback);


In [ ]:
model_jetChargedHadronMult.save('models/model_jetChargedHadronMult.h5')
model_jetChargedHadronMult.save_weights('models/model_mult_weights.h5')

In [ ]:

# Define the DNN architecture
model = Sequential()
model.add(Dense(100, kernel_initializer='normal', activation='relu', input_dim=train_x.shape[1]))
model.add(Dropout(0.2))
model.add(Dense(100, kernel_initializer='normal', activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(50, kernel_initializer='normal', activation='relu'))
model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
model.compile(optimizer='Nadam', loss='binary_crossentropy', metrics=['accuracy'])

# Weight the training samples so that there is equal weight on gluon and quark jets
# even if there are different amount of them in the training set
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(train_y), y=train_y[:])
print(class_weights)

callback=EarlyStopping(monitor='val_loss',min_delta=0.00001,patience=8,verbose=1,mode='auto',baseline=None,restore_best_weights=False)

# Train the model
model.fit(train_x,
        train_y,
        epochs=200,
        batch_size=128,
        class_weight={0:class_weights[0] , 1: class_weights[1]},
        validation_split=0.2,
        shuffle=True,
       verbose=1,callbacks=callback);

In [ ]:
model.save('models/model.h5')
model.save_weights('models/model_weights.h5')

Let's see how the trained model performs by first creating predictions for the test set and plotting the classifier output.

In [ ]:
pred_ptD = model_ptD.predict(test_x_QG_ptD)
pred_axis2=model_axis2.predict(test_x_QG_axis2)
pred_mult=model_mult.predict(test_x_QG_mult)
pred_jetPt=model_jetPt.predict(test_x_jetPt)
pred_jetGirth=model_jetGirth.predict(test_x_jetGirth)
pred_jetChargedHadronMult=model_jetChargedHadronMult.predict(test_x_jetChargedHadronMult)
pred_y=model.predict(test_x)


In [ ]:
plt.clf()
binnings = np.arange(0.0, 1.0, 0.04)
plt.hist( pred_y[test_y==0], bins=binnings, alpha=0.8, label="Gluons", density=1 )
plt.hist( pred_y[test_y==1], bins=binnings, alpha=0.8, label="Quarks", density=1 )
plt.xlim(0,1)
plt.legend()
plt.xlabel('DNN output value')
plt.title('Simple DNN classifier');

Calculating the Area under the curve (AUC) of the Receiver Operating Curve (ROC) to find the efficiency of our network.

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve, auc

fpr_dnn_ptD, tpr_dnn_ptD, thresholds_dnn_ptD  = roc_curve(test_y, pred_ptD)
roc_auc_ptD = auc(fpr_dnn_ptD, tpr_dnn_ptD)

fpr_dnn_axis2, tpr_dnn_axis2, thresholds_dnn_axis2  = roc_curve(test_y, pred_axis2)
roc_auc_axis2 = auc(fpr_dnn_axis2, tpr_dnn_axis2)

fpr_dnn_mult, tpr_dnn_mult, thresholds_dnn_mult  = roc_curve(test_y, pred_mult)
roc_auc_mult = auc(fpr_dnn_mult, tpr_dnn_mult)

fpr_dnn_jetpt, tpr_dnn_jetpt, thresholds_dnn_jetpt  = roc_curve(test_y, pred_jetPt)
roc_auc_jetpt = auc(fpr_dnn_jetpt, tpr_dnn_jetpt)

fpr_dnn_jetgirth, tpr_dnn_jetgirth, thresholds_dnn_jetgirth  = roc_curve(test_y, pred_jetGirth)
roc_auc_jetgirth = auc(fpr_dnn_jetgirth, tpr_dnn_jetgirth)


fpr_dnn_jetCHmult, tpr_dnn_jetCHmult, thresholds_dnn_jetCHmult  = roc_curve(test_y, pred_jetChargedHadronMult)
roc_auc_jetCHmult = auc(fpr_dnn_jetCHmult, tpr_dnn_jetCHmult)

fpr_dnn, tpr_dnn, thresholds_dnn  = roc_curve(test_y, pred_y)
roc_auc_dnn = auc(fpr_dnn, tpr_dnn)



In [ ]:


plt.clf()
plt.plot(fpr_dnn_ptD, tpr_dnn_ptD, 'b', label='ptD only, AUC = %0.4f'% roc_auc_ptD)
plt.plot(fpr_dnn_axis2, tpr_dnn_axis2,   'r', label='axis2 only, AUC = %0.4f'% roc_auc_axis2)
plt.plot(fpr_dnn_mult, tpr_dnn_mult,   'g', label='mult only, AUC = %0.4f'% roc_auc_mult)
plt.plot(fpr_dnn_jetgirth, tpr_dnn_jetgirth,   'y', label='jetgirth only, AUC = %0.4f'% roc_auc_jetgirth)
plt.plot(fpr_dnn_jetpt, tpr_dnn_jetpt,   'olive',label='jetpt only, AUC = %0.4f'% roc_auc_jetpt)
plt.plot(fpr_dnn_jetCHmult, tpr_dnn_jetCHmult,   'navy', label='jetCHmult only, AUC = %0.4f'% roc_auc_jetCHmult)
plt.plot(fpr_dnn, tpr_dnn, 'k', label='Simple DNN classifier, AUC = %0.4f'% roc_auc_dnn)
plt.plot([0,1], [0,1], 'k--')
plt.text(0.05, 0.01, '$0<|\eta|<2.5 ,$ $30 $ GeV$<p_T<600 $ GeV', fontsize = 10)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.legend(loc = 'lower right',prop={'size': 7})
plt.title("Receiver operating characteristic")
plt.ylabel('Quark jet acceptance rate')
plt.xlabel('Gluon jet acceptance rate')
